Copyright © 2022, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved. 
SPDX-License-Identifier: Apache-2.0

# Configure a Container Publishing Destination with a Git Repository

_**Note:** With the release of SAS Viya 2021.2.5, you can push SAS Container Runtime artifacts to Git when publishing objects to a container publishing destination._

Here are the artifacts that are included when publishing objects using SAS Container Runtime:

* Model scoring files and resources that are used to score new data with your trained models. 
* Dockerfile that contains the commands necessary to assemble the image. 
* Manifest file that contains information about the image. 

This information ties together the model within SAS Model Manager with the immutable container published within the container registry. 

This notebook example, walks you through the necessary steps to configure a Private Docker publishing destination and associate it with a Git repository. When a model is published using SAS Container Runtime, the container artifacts are also be pushed into the specified Git repository.

Complete the following steps:

1. Specify the necessary Python packages.
2. Specify user and connection Information.
3. Get an authorization token.
4. Create a domain and user credentials.
5. Submit an API post to create a Private Docker destination that is associated with a Git repository.


## Getting Started
Specify the necessary Python packages.

In [ ]:
import requests
import os, json
import getpass
import base64

## Specify User and Connection Information

In [ ]:
# Supplied SAS Viya Information 
hostport = "Your_Viya_Environment"
user = "Your_Viya_User"
password = "Your_Viya_Password"

# Create Domain 
domain_name = "Name_for_Domain"
domain_description = "Optional_Domain_Description"

# Create Destination 
destName = "Name_for_Destination"
destDescription = "Optional_Destination_Description"

# Supplied Container Information 
encoded_kubeKeyData = "Encoded_Kube_Key"
encoded_kubeCertData = "Encoded_Kube_Cert"
userId = "Docker_User_Id"
docker_password = "Docker_Password"
baseRepoUrl = "Base_Image_Respository_URL"
kubeUrl = "Docker_Host_URL"

# Supplied Git Information
gitUserId = "Your_Git_User"
gitAccessToken = "Access_Token_for_Git_User"
remoteRepoURL = "Destination_Git_Repo_URL"
localRepoPath = "Local_Git_Repo_Path"

userEmail = "Optional_Email"

## Get an Authorization Token and Define Headers

When calling APIs in SAS Viya, you need an authorization token. This code generates the authorization token.

In [ ]:
authUri="/SASLogon/oauth/token"

headersAuth={
    "accept":"application/json",
    "content-type":"application/x-www-form-urlencoded",
    "Authorization":"Basic c2FzLmVjOg=="
}

authBody='grant_type=password&username=' + user + '&password=' + password
authReturn = requests.post(hostport+authUri, data=authBody, headers=headersAuth, verify=False)
authToken = authReturn.json()['access_token']

# print(authToken)

## Create a Domain and User Credentials

Before you can create a container destination, you must create a domain and supply the necessary credentials to access the destination and the Git  repository.

In [ ]:
domains_uri = "/credentials/domains/"
credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    'Authorization': 'Bearer ' + authToken
}

my_domain_url = hostport + domains_uri + domain_name
domain_attrs = {
    "id":domain_name,
    "type":"base64",
    "description": domain_description
}

domain = requests.put(my_domain_url, 
                      data=json.dumps(domain_attrs), 
                      headers=credential_domain_headers, verify=False)

# print(domain)

In [ ]:
my_credential_url = my_domain_url + "/users/" + user

encoded_userId = str(base64.b64encode(userId.encode("utf-8")), "utf-8")
encoded_password = str(base64.b64encode(docker_password.encode("utf-8")), "utf-8")

encoded_gituserId = str(base64.b64encode(gitUserId.encode("utf-8")), "utf-8")
encoded_gitpassword = str(base64.b64encode(gitAccessToken.encode("utf-8")), "utf-8")

In [ ]:
credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    'Authorization': 'Bearer ' + authToken
}

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user,
    "domainType":"base64",
    "properties":{"dockerRegistryUserId":encoded_userId,
                 "gitUserId":encoded_gituserId},
    "secrets":{
        "dockerRegistryPasswd":encoded_password,
        "kubeKeyData":encoded_kubeKeyData,
        "kubeCertData":encoded_kubeCertData,
        "gitAccessToken":encoded_gitpassword
    }
}

#print(my_credential_url)
credential = requests.put(my_credential_url,
                          data=json.dumps(credential_attrs),
                          headers=credential_user_headers, verify=False)

print(credential)

## Create a Destination with an Associated Git Repository

Here is the final step to create the Private Docker publishing destination and associate it with a Git repository. 

In [ ]:
targetDestination={
    "name":destName,
    "destinationType":"privateDocker",
    "description":destDescription,
    "properties":[
        {"name": "credDomainId",
        "value": domain_name},
        {"name":"baseRepoUrl",
        "value":baseRepoUrl},
        {"name":"kubeUrl",
        "value": kubeUrl},
        {"name": "RemoteRepositoryURL",
        "value": remoteRepoURL},
        {"name": "LocalRepositoryLocation",
        "value": localRepoPath},
        {"name": "UserEmail",
        "value": userEmail}
    ]
}

headersPost = {
    "Content-Type":"application/vnd.sas.models.publishing.destination.privatedocker+json",
    'Authorization': 'Bearer ' + authToken
}

destination = requests.post(hostport + "/modelPublish/destinations", 
                       data=json.dumps(targetDestination), headers=headersPost, verify=False)
# print(destination)

***